In [2]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("./data/ntsb.db")


query = """
SELECT ev.ev_id,
       evt.meaning   AS ev_type,
       pl.regis_no,
       ev.ntsb_no,
       STRFTIME('%d/%m/%Y',date(ev_date)) as ev_date,
       ev.ev_time,
       ev.ev_city,
       ev.ev_country,
       ev.latitude,
       ev.longitude,
       -- aircraft
	   plt.meaning   AS acft_category,
       pl.acft_make,
       acft_model,
       pl.num_eng,
       dam.meaning   AS DAMAGE,
       tfly.meaning   AS flight_type,
       pl.afm_hrs,
       pl.date_last_insp,
       pl.afm_hrs_last_insp,	   
       --'https://data.ntsb.gov/Docket?ProjectID=' || substr(ev.ev_id,-6) as ntsb_docket,
       -- W & conditions
	   lcond.meaning AS light_condition,
       wmc.meaning   wmc,
       gust_ind,--wx_int_precip,
       on_ground_collision,
       -- injures
	   ev.inj_f_grnd,
       ev.inj_m_grnd,
       ev.inj_s_grnd,
       ev.inj_tot_f,
       ev.inj_tot_m,
       ev.inj_tot_n,
       ev.inj_tot_s,
       ev.inj_tot_t,
       ev_highest_injury,
       'calcular'    AS flight_phase,
       'calcular'    AS defining_event,
       case when inj_tot_t=inj_tot_f THEN 'F' ELSE 'T' END    AS survivors_b,
	0    AS pilot_flight_time_model,
       0    AS pilot_flight_time_total,
       'calcular'    AS pilot_flight_time_categ,
       'POST_1982'   AS SOURCE
FROM   EVENTS ev
       inner join aircraft pl
               ON ev.ev_id = pl.ev_id
       left join eadmspub_datadictionary evt
              ON ev.ev_type = evt.code_iaids
                 AND Trim(evt.ct_name) = 'ct_ev_type'
       left join eadmspub_datadictionary plt
              ON pl.acft_category = plt.code_iaids
                 AND Trim(plt.ct_name) = 'ct_acft_cat'
       left join eadmspub_datadictionary tfly
              ON pl.type_fly = tfly.code_iaids
                 AND Trim(tfly.ct_name) = 'ct_type_fly'				 
       left join ct_iaids dam
              ON pl.damage = dam.code_iaids
                 AND Trim(dam.ct_name) = 'ct_damage'
       left join ct_iaids lcond
              ON ev.light_cond = lcond.code_iaids
                 AND Trim(lcond.ct_name) = 'ct_light_cond'
       left join ct_iaids wmc
              ON ev.wx_cond_basic = wmc.code_iaids
                 AND Trim(wmc.ct_name) = 'ct_wx_cond_b'
where ev.ev_id not in('20081017X61856') -- vacío
;
"""


df = pd.read_sql_query(query, conn)


In [3]:
df.count()

ev_id                      30270
ev_type                    30270
regis_no                   30228
ntsb_no                    30270
ev_date                    30270
ev_time                    29885
ev_city                    30257
ev_country                 30266
latitude                   27088
longitude                  27090
acft_category              29591
acft_make                  30230
acft_model                 30219
num_eng                    25917
DAMAGE                     28263
flight_type                23346
afm_hrs                    19572
date_last_insp             19953
afm_hrs_last_insp           8277
light_condition            24896
wmc                        24923
gust_ind                   30261
on_ground_collision          809
inj_f_grnd                  2600
inj_m_grnd                  3190
inj_s_grnd                  2552
inj_tot_f                  30270
inj_tot_m                  30270
inj_tot_n                  30270
inj_tot_s                  30270
inj_tot_t 

In [3]:
df.to_csv("./data/ntsb_2008_2025.csv", index=False)
conn.close()

In [4]:
df.head(1000).to_csv("./data/ntsb_2008_2025_sample.csv", index=False)
conn.close()